<a href="https://colab.research.google.com/github/ndreeawan/Tubes_024-128/blob/main/tugas6_CBIR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d dionisiusdh/indonesian-batik-motifs

 98% 256M/262M [00:04<00:00, 64.1MB/s]
100% 262M/262M [00:04<00:00, 55.8MB/s]


In [4]:
!unzip indonesian-batik-motifs.zip

Archive:  indonesian-batik-motifs.zip
  inflating: batik-bali/1.jpg        
  inflating: batik-bali/10.jpg       
  inflating: batik-bali/11.jpg       
  inflating: batik-bali/12.jpg       
  inflating: batik-bali/13.jpg       
  inflating: batik-bali/14.jpg       
  inflating: batik-bali/15.jpg       
  inflating: batik-bali/16.jpg       
  inflating: batik-bali/17.jpg       
  inflating: batik-bali/18.jpg       
  inflating: batik-bali/19.jpg       
  inflating: batik-bali/2.jpg        
  inflating: batik-bali/20.jpg       
  inflating: batik-bali/21.jpg       
  inflating: batik-bali/22.jpg       
  inflating: batik-bali/23.jpg       
  inflating: batik-bali/24.jpg       
  inflating: batik-bali/25.jpg       
  inflating: batik-bali/26.jpg       
  inflating: batik-bali/27.jpg       
  inflating: batik-bali/28.jpg       
  inflating: batik-bali/29.jpg       
  inflating: batik-bali/3.jpg        
  inflating: batik-bali/30.jpg       
  inflating: batik-bali/31.jpg       
  inflating:

In [5]:
!mkdir '/content/batik-datasets'

In [8]:
!mv "/content/batik-bali/" "/content/batik-datasets/"
!mv "/content/batik-garutan/" "/content/batik-datasets/"
!mv "/content/batik-keraton/" "/content/batik-datasets/"

mv: cannot stat '/content/batik-bali/': No such file or directory
mv: cannot stat '/content/batik-garutan/': No such file or directory
mv: cannot stat '/content/batik-keraton/': No such file or directory


In [25]:
!rm -rf batik-megamendung
!rm -rf batik-parang
!rm -rf batik-pekalongan
!rm -rf batik-priangan
!rm -rf batik-sekar
!rm -rf batik-sidoluhur
!rm -rf batik-sidomukti

In [7]:
!pip install split-folders

In [9]:
!mkdir batik-fix

In [10]:
import splitfolders

base_dir = "/content/batik-datasets/"
splitfolders.ratio(base_dir, output="batik-fix", ratio=(.8, .1, .1))

Copying files: 150 files [00:00, 1566.52 files/s]


In [11]:
from imutils import paths
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

imagePathsTrain = list(paths.list_images("/content/batik-fix/train"))
trainX = []
trainY = []

for imagePath in imagePathsTrain:

	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(120, 120))
	image = img_to_array(image)
	image = preprocess_input(image)
	trainX.append(image)
	trainY.append(label)

trainX = np.array(trainX, dtype="float32")
trainY = np.array(trainY)

lb = LabelEncoder()
trainY = lb.fit_transform(trainY)
trainY = to_categorical(trainY)

In [12]:
imagePathsTest = list(paths.list_images("/content/batik-fix/test"))
testX = []
testY = []

for imagePath in imagePathsTest:

	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(120, 120))
	image = img_to_array(image)
	image = preprocess_input(image)
	testX.append(image)
	testY.append(label)

testX = np.array(testX, dtype="float32")
testY = np.array(testY)

lb = LabelEncoder()
testY = lb.fit_transform(testY)
testY = to_categorical(testY)

In [13]:
imagePathsTest = list(paths.list_images("/content/batik-fix/val"))
valX = []
valY = []

for imagePath in imagePathsTest:

	label = imagePath.split(os.path.sep)[-2]
	image = load_img(imagePath, target_size=(120, 120))
	image = img_to_array(image)
	image = preprocess_input(image)
	valX.append(image)
	valY.append(label)

valX = np.array(valX, dtype="float32")
valY = np.array(valY)

lb = LabelEncoder()
valY = lb.fit_transform(valY)
valY = to_categorical(valY)

In [14]:
print("SHAPE OF TRAIN FEATURES: "+str(trainX.shape))
print("SHAPE OF TRAIN LABELS: "+str(trainY.shape))
print("SHAPE OF TEST FEATURES: "+str(testX.shape))
print("SHAPE OF TEST LABELS: "+str(testY.shape))
print("SHAPE OF VALIDATION FEATURES: "+str(valX.shape))
print("SHAPE OF VALIDATION LABELS: "+str(valY.shape))

SHAPE OF TRAIN FEATURES: (120, 120, 120, 3)
SHAPE OF TRAIN LABELS: (120, 3)
SHAPE OF TEST FEATURES: (15, 120, 120, 3)
SHAPE OF TEST LABELS: (15, 3)
SHAPE OF VALIDATION FEATURES: (15, 120, 120, 3)
SHAPE OF VALIDATION LABELS: (15, 3)


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [20]:
INIT_LR = 1e-4
EPOCHS = 50
BS = 32

In [21]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(120, 120, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = True

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
HR = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(valX, valY),
	validation_steps=len(valX) // BS,
	epochs=EPOCHS)

Epoch 1/50
3/3 [==============================] - 25s 7s/step - loss: 1.5981 - accuracy: 0.3409
Epoch 2/50
3/3 [==============================] - 21s 7s/step - loss: 1.2668 - accuracy: 0.4688
Epoch 3/50
3/3 [==============================] - 19s 6s/step - loss: 1.0535 - accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 19s 7s/step - loss: 0.9492 - accuracy: 0.5455
Epoch 5/50
3/3 [==============================] - 20s 7s/step - loss: 0.9867 - accuracy: 0.5417
Epoch 6/50
3/3 [==============================] - 19s 6s/step - loss: 0.8880 - accuracy: 0.5795
Epoch 7/50
3/3 [==============================] - 19s 6s/step - loss: 0.6817 - accuracy: 0.6705
Epoch 8/50
3/3 [==============================] - 19s 6s/step - loss: 0.7667 - accuracy: 0.6818
Epoch 9/50
3/3 [==============================] - 20s 7s/step - loss: 0.5373 - accuracy: 0.7812
Epoch 10/50
3/3 [==============================] - 19s 6s/step - loss: 0.5865 - accuracy: 0.7500
Epoch 11/50
3/3 [======================

In [19]:
from sklearn.metrics import classification_report

y_pred = np.around(model.predict(testX))
print(classification_report(testY, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.62      1.00      0.77         5
           2       1.00      0.20      0.33         5

   micro avg       0.62      0.53      0.57        15
   macro avg       0.71      0.53      0.52        15
weighted avg       0.71      0.53      0.52        15
 samples avg       0.53      0.53      0.53        15



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
